# Module Feedback

This system records the responses of students on their learning experience at university.

![rel](https://sqlzoo.net/w/images/7/71/Module-feedback.png)

In [ ]:
import os
import pandas as pd
import findspark
os.environ['SPARK_HOME'] =  '/opt/spark'
findspark.init()

from pyspark.sql import SparkSession
ss = (SparkSession.builder.appName('app00')
      .config('spark.sql.warehouse.dir', 'hdfs://quickstart.cloudera:8020/user/hive/warehouse')
      .config('hive.metastore.uris', 'thrift://quickstart.cloudera:9083')
      .enableHiveSupport().getOrCreate())

## 1. Find the student name from a matriculation number

**Find the name of the student with number 50200100**

## 2. Find the modules studied by a student

**Show the module code and module name for modules studied by the student with number 50200100 in session 2016/7 TR1**

## 3. Find the modules and module leader studied by a student

**Show the module code and module name and details of the module leader for modules studied by the student with number 50200100 in session 2016/7 TR1**

## 4. Show the scores for module SET08108

**Show the Percentage of students who gave 4 or 5 to module SET08108 in session 2016/7 TR1**

(note that this is not real data, these responses were randomly generated)

## 5. Show the frequency chart for module SET08108 for question 4.1

**For each response 1-5 show the number of students who gave that response (Module SET08108, 2016/7, TR1)**

(note that this is not real data, these responses were randomly generated)